In [1]:
import os
import pandas as pd
import numpy as np
import re
from metadata_processing import compute_mask

In [2]:
# path to metadata
path_to_recordings_metadata = '/home/nakedmoleratvoices/Mole rats reborn/CodeRefactoredFinal/data/'
recordings_metadata_name = 'recordings_metadata.csv'
recs_metadata = pd.read_csv(path_to_recordings_metadata + recordings_metadata_name)
recs_metadata['processing stage'].value_counts()

traced and checked     1045
labeled and checked     289
traced                   90
labeled                  22
fresh                    16
Name: processing stage, dtype: int64

### Fresh

In [3]:
recs_to_check = recs_metadata[recs_metadata['processing stage'] == 'fresh']
print('Total %d fresh' % recs_to_check.shape[0])
recs_to_check.head()

print('Experiemnts:\n', dict(recs_to_check['experiment'].value_counts()))

Total 16 fresh
Experiemnts:
 {'stark epochs': 8, 'single rat softchirps;baratheon epochs': 2, 'single rat softchirps': 6}


In [4]:
for rec_name, rec_path in recs_to_check[['name', 'path']].values:
    if not os.path.isfile(rec_path + rec_name):
        print('Rec %s was not found in the location described by path')
    elif os.path.isfile(rec_path + rec_name[:-3] + 'txt'):
        print('Txt file exists for %s' % rec_name)

### Checking splits

In [36]:
# checking that split and split and checked recordings all have txt and only classes in txts are noise and sound

In [5]:
stage = 'labeled'
recs_to_check = recs_metadata[recs_metadata['processing stage'] == stage]
#recs_to_check = recs_to_check[recs_to_check['experiment'] == 'stark epochs']
print('Total %d %s' % (recs_to_check.shape[0], stage))
print('Experiemnts:\n', dict(recs_to_check['experiment'].value_counts()))
print('Stages:\n', dict(recs_to_check['processing stage'].value_counts()))

Total 22 labeled
Experiemnts:
 {'single rat softchirps;baratheon epochs': 22}
Stages:
 {'labeled': 22}


In [6]:
recs_with_zero_splits = []
sounds = set()
for stage in set(recs_metadata['processing stage']):
    if stage == 'fresh':
        continue
    recs_to_check = recs_metadata[recs_metadata['processing stage'] == stage]
    for rec_name, rec_path in recs_to_check[['name', 'path']].values:
        df_txt = pd.read_csv(rec_path + rec_name[:-3] + 'txt', sep = '\t', )
        if len(df_txt) == 0:
            print(rec_name, 'zero splits')
            recs_with_zero_splits.append(rec_name)
            continue
        rec = np.load(rec_path + rec_name)
        max_e = int(22050 * np.max(df_txt['e']))
        assert max_e <= len(rec), 'Sound frame is beyond rec'
        if 'sound' in set(df_txt['cl']) or ('soundunconf' in set(df_txt['cl'])):
            print('Recording %s has class %s in in txt' %(rec_name, cl))
        sounds = sounds | set(df_txt['cl'])

In [169]:
for rec_name in recs_with_zero_splits:
    ind = np.where(recs_metadata['name'] == rec_name)[0][0]
    rec_path = recs_metadata['path'].iloc[ind]
    df_txt = pd.read_csv(rec_path + rec_name[:-3] + 'txt', sep = '\t', )
    print(rec_name, len(df_txt))
    recs_metadata['processing stage'].iloc[ind] = 'fresh'
    os.remove(rec_path + rec_name[:-3] + 'txt')
recs_metadata.to_csv(path_to_recordings_metadata + recordings_metadata_name, index = None)

baratheon_04-04-19_9440_0000001.npy 0
baratheon_04-04-19_0855_0000001.npy 0
baratheon_04-04-19_2137_0000001.npy 0
baratheon_04-04-19_9466_0000001.npy 0
baratheon_04-04-19_9452_0000001.npy 0
baratheon_04-04-19_2141_0000001.npy 0
stark_15-05-18_1549_0000019.npy 0


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### Traces

In [67]:
# check if there are traces for recordings with stage != traced or traced and checked

In [7]:
recs_not_in = set()
traces_not_in = []
recs_wrong_status = set()

rec_inds = {}
for path in set(recs_metadata['path']):
    path_traces = path.replace('recordings', 'traces')
    traces = os.listdir(path_traces)
    for t in traces:
        rec_name = re.findall('[0-9]_[a-z]+_.+', t)[0][2:-3] + 'npy'
        if rec_name not in recs_metadata['name'].values:
            print(rec_name, 'not in ')
            recs_not_in.add(rec_name)
            traces_not_in.append(t)
            continue
        if rec_name in rec_inds:
            rec_ind = rec_inds[rec_name]
        else:
            rec_ind = np.where(recs_metadata['name'] == rec_name)[0][0]
            rec_inds[rec_name] = rec_ind
        stage = recs_metadata['processing stage'].iloc[rec_ind]
        if 'traced' not in stage:
            print(rec_name, 'wrong stage %s' % stage)
            recs_wrong_status.add(rec_name)
        

In [11]:
recs_not_in

set()

In [120]:
base = '/home/nakedmoleratvoices/Mole rats reborn/CodeRefactoredFinal/data/'
for rec_name in recs_not_in:
    colony = rec_name[: rec_name.find('_')]
    rec_path =  base + colony + '/recordings/'
    if os.path.isfile(rec_path + rec_name):
        print('isrec', rec_name)
    if os.path.isfile(rec_path + rec_name[:-3] + 'txt'):
        print('istxt', rec_name)
    

In [182]:
traces_status = {rn : [0, 0] for rn in recs_wrong_status}
for rec_name in recs_wrong_status:
    rec_ind = np.where(recs_metadata['name'] == rec_name)[0][0]
    rec_path = recs_metadata['path'].iloc[rec_ind]
    rec_stage = recs_metadata['processing stage'].iloc[rec_ind]
    #df_txt = pd.read_csv(rec_path + rec_name[:-3] + 'txt', sep = '\t', )
    rec_traces = []
    for t in os.listdir(rec_path.replace('recordings', 'traces')):
        if rec_name[:-3] not in t:
            continue
        rec_traces.append(t)
        is_fresh = 1 if 'fresh' in t else 0
        if is_fresh:
            traces_status[rec_name][0] +=1 
        else:
            traces_status[rec_name][1] += 1

In [190]:
for rec_name in recs_wrong_status:
    rec_ind = np.where(recs_metadata['name'] == rec_name)[0][0]
    rec_path = recs_metadata['path'].iloc[rec_ind]
    rec_traces = [t for t in os.listdir(rec_path.replace('recordings', 'traces')) if rec_name[:-3] in t]
    for t in rec_traces:
        os.remove(rec_path.replace('recordings', 'traces') + t)
#recs_metadata.to_csv(path_to_recordings_metadata + recordings_metadata_name, index = None)

In [281]:
stage = 'traced and checked'
recs_to_check = recs_metadata[recs_metadata['processing stage'] == stage]
print('Total %d %s' % (recs_to_check.shape[0], stage))
print('Experiemnts:\n', dict(recs_to_check['experiment'].value_counts()))
print('Stages:\n', dict(recs_to_check['processing stage'].value_counts()))
print('Colonies:\n', dict(recs_to_check['colony'].value_counts()))

Total 1045 traced and checked
Experiemnts:
 {'single rat softchirps': 426, 'single rat softchirps;baratheon epochs': 9, 'single rat softchirps;deafening': 314, 'stark epochs;classification': 89, 'stark epochs': 166, 'foster pups tracking': 38, 'single rat softchirps;foster pups tracking': 3}
Stages:
 {'traced and checked': 1045}
Colonies:
 {'stark': 319, 'freefolka': 25, 'arryn': 40, 'martell': 75, 'freefolkb': 16, 'playback': 29, 'baratheon': 85, 'dothrakib': 25, 'dothrakia': 18, 'targaryen': 99, 'lannister': 314}


In [351]:
# checks each trace has entry in the txt file
stop = False

bad_traces = []
for rec_name in bad_recs:
    rec_path = recs_metadata['path'].iloc[np.where(recs_metadata['name'] == rec_name)[0][0]]
    try:
        df_txt = txts_dict[rec_name]
    except:
        print('except', rec_name)
        df_txt = pd.read_csv(rec_path + rec_name[:-3] + 'txt', sep = '\t')
    rec_traces = [t for t in os.listdir(rec_path.replace('recordings', 'traces')) if rec_name[:-3] in t]
    inds_to_keep = []
    for t in rec_traces:
        s, e = tuple(map(float, re.findall('[0-9]+\.[0-9]+\_[0-9]+\.[0-9]+', t)[0].split('_'))) 
        closest_ind =  np.argmin(np.abs(df_txt['s'] - s))
        inds_to_keep.append(closest_ind)
        s_real, e_real = df_txt[['s', 'e']].iloc[closest_ind].values
        
        if e_real <= s or e <= s_real:
            overlap = 0
        else:
            overlap = (min(e_real, e) - max(s_real, s)) / (max(e_real, e) - min(s_real, s))
        if overlap <= 0.9:
            print(t, 'not in txt')
            bad_traces.append(t)
        cl = df_txt['cl'].iloc[closest_ind]
    #df_txt = df_txt.iloc[sorted(inds_to_keep)]
    #df_txt['cl'] = 'softchirp'
    #df_txt.to_csv(rec_path + rec_name[:-3] + 'txt', sep = '\t', index = None)

trace_sound_12.761043083900226_13.058321995464853_stark_31-03-18_9597_0000085.png not in txt
trace_sound_16.62077097505669_16.86004535147392_stark_31-03-18_9597_0000085.png not in txt
trace_sound_14.16390022675737_14.405986394557823_stark_31-03-18_9597_0000085.png not in txt
trace_sound_15.416009070294784_15.658367346938775_stark_31-03-18_9597_0000085.png not in txt
trace_sound_18.62108843537415_18.859773242630386_stark_31-03-18_1556_0000074.png not in txt
trace_sound_17.0831746031746_17.3772335600907_stark_31-03-18_1556_0000074.png not in txt
trace_sound_22.661360544217686_22.896961451247165_stark_31-03-18_1556_0000074.png not in txt
trace_sound_104.76077097505669_104.95709750566894_stark_05-09-17_8318_0000006.png not in txt
trace_sound_140.70657596371882_140.9091609977324_stark_05-09-17_8318_0000006.png not in txt
trace_sound_114.6514739229025_114.7598185941043_stark_05-09-17_8318_0000006.png not in txt
trace_sound_135.68648526077098_135.84625850340137_stark_05-09-17_8318_0000006.png

trace_sound_103.51297052154194_103.70784580498866_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_115.71918367346939_115.9702947845805_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_133.87537414965988_134.1724716553288_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_137.31990929705216_137.52004535147393_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_118.93179138321996_119.17351473922902_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_131.78585034013605_132.08272108843536_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_103.09160997732427_103.28335600907029_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_110.42920634920635_110.62222222222222_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_114.88362811791383_115.08770975056689_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_106.57931972789116_106.72453514739229_stark_31-03-18_9660_9551_0000098.png not in txt
trace_sound_139.356054

trace_sound_555.0848072562359_555.381179138322_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_630.0426757369614_630.2867120181406_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_590.3795918367347_590.6699773242631_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_604.2660770975057_604.6030385487528_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_527.3609070294784_527.6533786848073_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_632.691201814059_632.8926077097506_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_538.6468480725623_538.8531065759637_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_557.1365079365079_557.418820861678_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_484.5444444444444_484.7505215419501_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_662.0439002267574_662.332380952381_stark_05-10-17_8322_9660_0000004.png not in txt
trace_sound_478.2740589569161_478.4725623582

trace_sound_16.987573696145123_17.182766439909297_stark_01-11-17_8322_0000033.png not in txt
trace_sound_19.31605442176871_19.55251700680272_stark_01-11-17_8322_0000033.png not in txt
trace_sound_15.603310657596372_15.796054421768707_stark_01-11-17_8322_0000033.png not in txt
trace_sound_25.820090702947844_26.010068027210885_stark_05-09-17_9453_0000015.png not in txt
trace_sound_28.503990929705214_28.656507936507936_stark_05-09-17_9453_0000015.png not in txt
trace_sound_26.97043083900227_27.173333333333332_stark_05-09-17_9453_0000015.png not in txt
trace_sound_23.534965986394557_23.695419501133788_stark_05-09-17_9453_0000015.png not in txt
trace_sound_60.41079365079365_60.557596371882084_stark_19-06-18_8406_0000017.png not in txt
trace_sound_57.34961451247165_57.596916099773246_stark_19-06-18_8406_0000017.png not in txt
trace_sound_13.227165532879818_13.475963718820863_stark_31-03-18_9462_0000075.png not in txt
trace_sound_11.87968253968254_12.128163265306123_stark_31-03-18_9462_000007

trace_sound_13.372154195011339_13.614557823129251_stark_06-02-18_8304_0000017.png not in txt
trace_sound_232.46757369614514_232.6734693877551_stark_06-02-18_8406_8529_0000014.png not in txt
trace_sound_258.436462585034_258.63519274376415_stark_06-02-18_8406_8529_0000014.png not in txt
trace_sound_221.23092970521543_221.52113378684808_stark_06-02-18_8406_8529_0000014.png not in txt
trace_sound_289.0878911564626_289.3212244897959_stark_06-02-18_8406_8529_0000014.png not in txt
trace_sound_254.76403628117913_254.96068027210885_stark_06-02-18_8406_8529_0000014.png not in txt
trace_sound_244.96149659863946_245.15886621315192_stark_06-02-18_8406_8529_0000014.png not in txt
trace_sound_212.0408163265306_212.28104308390022_stark_06-02-18_8406_8529_0000014.png not in txt
trace_sound_220.81097505668933_221.10272108843537_stark_06-02-18_8406_8529_0000014.png not in txt
trace_sound_219.0497052154195_219.34421768707483_stark_06-02-18_8406_8529_0000014.png not in txt
trace_sound_229.08544217687074_2

...

trace_sound_287.82321995464855_288.02208616780047_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_268.2352380952381_268.46920634920633_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_318.80104308390025_318.99927437641725_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_267.1119274376417_267.3082993197279_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_232.5661224489796_232.76689342403628_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_234.93650793650792_235.08426303854876_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_258.71160997732426_258.950022675737_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_272.22956916099776_272.37378684807254_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_245.47102040816327_245.67265306122448_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_238.97551020408164_239.21324263038548_stark_19-06-18_9453_8406_0000023.png not in txt
trace_sound_286.7643990929

trace_sound_118.55142857142857_118.79451247165532_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_95.94140589569162_96.13972789115647_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_116.69968253968254_116.89297052154195_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_99.97532879818594_100.21913832199546_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_130.0247619047619_130.21868480725624_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_125.659410430839_125.90140589569161_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_115.30648526077097_115.4572335600907_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_110.75183673469388_110.94820861678005_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_119.39560090702948_119.58816326530612_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_102.49083900226758_102.68421768707483_stark_27-11-17_9453_9551_0000015.png not in txt
trace_sound_91.570158730158

trace_sound_1439.3517006802722_1439.550612244898_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1538.0373242630385_1538.2295691609977_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1355.8019954648526_1355.909433106576_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1171.208299319728_1171.4026757369616_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1559.0273922902495_1559.2200907029478_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1593.8139229024944_1594.013514739229_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1365.698820861678_1365.8479818594105_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1482.1186394557824_1482.2719727891156_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1420.7804535147393_1421.0149659863946_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1355.5724716553289_1355.675328798186_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1562.279954648

trace_sound_1459.5580498866213_1459.7020408163266_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1121.1002721088435_1121.2526077097505_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1234.4580498866212_1234.6524263038548_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1111.8091609977325_1112.0086621315193_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1594.0949206349205_1594.2443083900228_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1545.1891156462584_1545.3392743764173_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1241.2872108843537_1241.3953741496598_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1248.5749206349205_1248.769433106576_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1424.265396825397_1424.4059863945579_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1427.8349659863945_1427.9868027210885_stark_21-08-17_9453_9474_0000027.png not in txt
trace_sound_1191.83265

trace_sound_27.342494331065758_27.500544217687075_stark_15-05-18_8406_0000039.png not in txt
trace_sound_29.665396825396826_29.82530612244898_stark_01-11-17_9478_0000030.png not in txt
trace_sound_45.54639455782313_45.707074829931976_stark_05-10-17_9453_9551_0000015.png not in txt
trace_sound_40.20630385487528_40.36086167800453_stark_05-10-17_9453_9551_0000015.png not in txt
trace_sound_54.09972789115646_54.24367346938776_stark_05-10-17_9453_9551_0000015.png not in txt
trace_sound_51.0784126984127_51.272063492063495_stark_05-10-17_9453_9551_0000015.png not in txt
trace_sound_39.657868480725625_39.80367346938775_stark_05-10-17_9453_9551_0000015.png not in txt
trace_sound_44.48235827664399_44.67836734693878_stark_05-10-17_9453_9551_0000015.png not in txt
trace_sound_41.60825396825397_41.80390022675737_stark_05-10-17_9453_9551_0000015.png not in txt
trace_sound_43.788662131519274_43.98530612244898_stark_05-10-17_9453_9551_0000015.png not in txt
trace_sound_43.417142857142856_43.6560544217

In [352]:
len(bad_traces)

6603

In [346]:
rec_path = '/home/nakedmoleratvoices/Mole rats reborn/CodeRefactoredFinal/data/stark/recordings/'
bad_recs = list(set([re.findall('[0-9]_[a-z]+_.+', t)[0][2:-3] + 'npy' for t in bad_traces]))
recs_dict = {rn : np.load(rec_path + rn) for rn in bad_recs}
txts_dict = {rn : pd.read_csv(rec_path + rn[:-3] + 'txt', sep = '\t',) for rn in bad_recs}

In [347]:
num_bad = 0
num_fresh = 0
num_added = 0

to_delete = []


for t in bad_traces:
    s, e = tuple(map(lambda x: int(22050 * float(x)), re.findall('[0-9]+\.[0-9]+\_[0-9]+\.[0-9]+', t)[0].split('_'))) 
    rec_name = re.findall('[0-9]_[a-z]+_.+', t)[0][2:-3] + 'npy'
    rec = recs_dict[rec_name]

    if len(rec) <= e:
        to_delete.append(t)
        num_bad += 1
        continue
        
    if 'fresh' in t or 'unconf' in t:
        num_fresh += 1
    else:
        new_cl = 'softchirp'
        new_t = t.replace('sound', new_cl)
        txts_dict[rec_name] = txts_dict[rec_name].append(pd.DataFrame([[s/22050, e/22050, new_cl]], 
                                                                      columns = ['s', 'e',
                                                                                 'cl'])).reset_index(drop = True)
        num_added += 1
        

print(num_bad, num_fresh, num_added)

6603 0 15438


In [362]:
for rec_name in txts_dict:
    txt = txts_dict[rec_name].sort_values(by = 's')
    txt.to_csv(rec_path + rec_name[:-3] + 'txt', index = None, sep ='\t')